In [1]:
import pandas as pd
import numpy as np
import json
from collections import namedtuple
import pickle

In [2]:
df = pd.read_csv('../data/one_travel_chain_time.csv')

unique_grid_ids = pd.unique(df[['grid_id_o', 'grid_id_d']].values.ravel('K'))

# 使用 factorize 方法为所有不同的 grid IDs 分配新的编码
new_codes, unique = pd.factorize(unique_grid_ids, sort=True)

# 创建一个字典来映射旧的 grid IDs 到新的编码
grid_id_mapping = dict(zip(unique, new_codes))

df['grid_new_o'] = df['grid_id_o'].map(grid_id_mapping)
df['grid_new_d'] = df['grid_id_d'].map(grid_id_mapping)
df.head()

,who,date,seiqd,mode,lambda_o,phi_o,lambda_d,phi_d,migrt,weekend,...,LU_Special,LU_Transportation,LU_Wild,trip_routelen,grid_id_o,grid_id_d,stime,etime,grid_new_o,grid_new_d
0,36384703,20190102,1,1,114.033830,22.621549,114.064399,22.554032,20190706,False,...,0.0,0.000000,0.002857,8885.289,27962.0,17265,2019-01-02 08:33:19,2019-01-02 08:41:34,12,319
1,36384703,20190102,2,1,114.064399,22.554032,114.041782,22.544709,20190706,False,...,0.0,0.000000,0.027380,3314.763,17265.0,15470,2019-01-02 09:55:19,2019-01-02 09:58:23,319,285
2,36384703,20190102,3,1,114.041782,22.544709,114.033830,22.621549,20190706,False,...,0.0,0.241541,0.237051,11424.598,15470.0,27962,2019-01-02 11:06:57,2019-01-02 11:28:24,285,12
3,36384703,20190103,1,1,114.033830,22.621549,114.064399,22.554032,20190706,False,...,0.0,0.000000,0.002857,8885.289,27962.0,17265,2019-01-03 08:22:40,2019-01-03 08:30:45,12,319
4,36384703,20190103,2,1,114.064399,22.554032,114.041782,22.544709,20190706,False,...,0.0,0.000000,0.027380,3314.763,17265.0,15470,2019-01-03 09:39:25,2019-01-03 09:42:29,319,285


In [3]:
id_fnid_mapping = dict(zip(new_codes,unique))
with open("../data/id_fnid_mapping.pkl", "wb") as f:
    pickle.dump(id_fnid_mapping, f)

In [4]:
def build_chain(group):
    chains = []
    current_chain = [group['grid_id_o'].iloc[0]]
    
    for i in range(len(group) - 1):
        current_chain.append(group['grid_id_d'].iloc[i])
        # Check if the next 'o' is different from the current 'd'
        if group['grid_id_d'].iloc[i] != group['grid_id_o'].iloc[i + 1]:
            chains.append(current_chain)
            current_chain = [group['grid_id_o'].iloc[i + 1]]
    
    # Append the last destination and the final chain
    current_chain.append(group['grid_id_d'].iloc[-1])
    chains.append(current_chain)
    
    return chains

def build_id_chain(group):
    chains = []
    current_chain = [group['grid_new_o'].iloc[0]]
    
    for i in range(len(group) - 1):
        current_chain.append(group['grid_new_d'].iloc[i])
        # Check if the next 'o' is different from the current 'd'
        if group['grid_new_d'].iloc[i] != group['grid_new_o'].iloc[i + 1]:
            chains.append(current_chain)
            current_chain = [group['grid_new_o'].iloc[i + 1]]
    
    # Append the last destination and the final chain
    current_chain.append(group['grid_new_d'].iloc[-1])
    chains.append(current_chain)
    
    return chains

def build_both_chains(group):
    travel_chain = build_chain(group)
    id_chain = build_id_chain(group)
    return pd.Series({'travel_chain': travel_chain, 'id_chain': id_chain})

In [5]:
# Group by 'date' and apply the chain-building function
grouped1 = df[df['date']<df['migrt']]
grouped2 = df[df['date']>df['migrt']]
grouped1 = grouped1.groupby('date').apply(build_both_chains).reset_index()
grouped2 = grouped2.groupby('date').apply(build_both_chains).reset_index()

In [6]:
# Define the namedtuple type
TravelData = namedtuple('TravelChain', ['date', 'travel_chain','id_chain'])

In [7]:
# Convert each row to a namedtuple
namedtuples_list1 = [TravelData(row.date, row.travel_chain[0], row.id_chain[0]) for _, row in grouped1.iterrows()]
namedtuples_list2 = [TravelData(row.date, row.travel_chain[0], row.id_chain[0]) for _, row in grouped2.iterrows()]
namedtuples_all = namedtuples_list1+namedtuples_list2

In [8]:
def int64_converter(obj):
    if isinstance(obj, np.int64):
        return int(obj)
    raise TypeError

dicts_list1 = [nt._asdict() for nt in namedtuples_list1]
with open('../data/before_migrt.json', 'w') as file:
    json.dump(dicts_list1, file, indent=4,default=int64_converter)

dicts_list2 = [nt._asdict() for nt in namedtuples_list2]
with open('../data/after_migrt.json', 'w') as file:
    json.dump(dicts_list2, file, indent=4,default=int64_converter)

dicts_list_all = [nt._asdict() for nt in namedtuples_all]
with open('../data/all_traj.json', 'w') as file:
    json.dump(dicts_list_all, file, indent=4,default=int64_converter)

In [9]:
with open('../data/all_traj.json', 'r') as file:
    loaded_dicts_all = json.load(file)
loaded_namedtuples_all = [TravelData(**d) for d in loaded_dicts_all]

In [10]:
loaded_namedtuples_all

[TravelChain(date=20190102, travel_chain=[27962.0, 17265, 15470, 27962], id_chain=[12, 319, 285, 12]),
 TravelChain(date=20190103, travel_chain=[27962.0, 17265, 15470, 27962], id_chain=[12, 319, 285, 12]),
 TravelChain(date=20190104, travel_chain=[27962.0, 15470, 27962, 22184, 19680], id_chain=[12, 285, 12, 95, 173]),
 TravelChain(date=20190105, travel_chain=[19680.0, 21884, 27962, 19769, 17994, 27962], id_chain=[173, 160, 12, 225, 188, 12]),
 TravelChain(date=20190107, travel_chain=[27962.0, 15470, 27962, 21502, 27962], id_chain=[12, 285, 12, 126, 12]),
 TravelChain(date=20190108, travel_chain=[27962.0, 15470, 27962, 21502], id_chain=[12, 285, 12, 126]),
 TravelChain(date=20190109, travel_chain=[21502.0, 27962, 17265, 27962], id_chain=[126, 12, 319, 12]),
 TravelChain(date=20190110, travel_chain=[27962.0, 15470, 21544, 27962], id_chain=[12, 285, 154, 12]),
 TravelChain(date=20190111, travel_chain=[27962.0, 15470, 27962], id_chain=[12, 285, 12]),
 TravelChain(date=20190112, travel_chai